In [1]:
import numpy as np
import numpy.random as rand
import pandas as pd
import datetime as dt

In [2]:
class SeminarSched:
    def __init__(self, personlist, taglist, rolelist, startdate='2020-1-20', enddate='2020-5-13', rand_seed=314):
        """
        Seminar schedule generator
        
        -----------
        Parameters
        -----------
        personlist : array_like
                     List of names as list/NumPy array.
        taglist : array_like
                    List of person cateogries as a list/NumPy array.
        rolelist : Pandas series
                   List of roles as a Pandas Series array.
        startdate : str
                    Start date in string form 'YYYY-M-D', integers only, separated by dashes, no leading zeros, no spaces.
        enddate : str
                  End date in the string form 'YYYY-M-D', integers only, separated by dashes, no leading zeros, no spaces.
        rand_seed : int
                    Seed for the random number generator.  
        
        """
        rand.seed(rand_seed)
        sdl = np.array(startdate.split('-')).astype(int)
        edl = np.array(enddate.split('-')).astype(int)
        start = dt.datetime(*sdl)
        end = dt.datetime(*edl)
        self.dates = pd.date_range(start, end, freq='W-MON').strftime('%d-%b')
        self.mems = personlist
        self.tags = taglist
        self.num_senior = len(np.where(self.tags == 'Senior')[0])
        self.num_junior = len(np.where(self.tags == 'Junior')[0])
        self.juniors = np.array([self.mems[i] for i in range(len(self.mems)) if self.tags[i] == 'Junior'])
        self.seniors = np.array([self.mems[i] for i in range(len(self.mems)) if self.tags[i] == 'Senior'])
        self.seniors = np.delete(self.seniors, np.where(self.seniors == 'Jing')[0])
        self.seniors = np.delete(self.seniors, np.where(self.seniors == 'Atchong')[0])
        self.roles = rolelist
        self.org = np.zeros((len(self.roles), len(self.dates)), dtype='<U16')
        self.org[0,0] = 'Jing'
        self.org[1,0] = 'Atchong'
        self.org[2,0] = 'Rhei'
        self.org[3,0] = 'LJ'
        self.org[4,0] = 'Andy'
        self.org[5,0] = 'Jara'
        self.org[6,0] = 'Clauzette'
        self.org[7,0] = 'Yego'
        self.org[-1,0] = 'None'
        self.org[-2,0] = 'None'
        self.org[0,1] = 'Imang'
        self.org[1,1] = 'Jai'
        self.org[2,1] = 'Raymond'
        self.org[3,1] = 'Jayson'
        self.org[4,1] = 'Elexis'
        self.org[5,1] = 'Jholeeh'
        self.org[6,1] = 'Micholo'
        self.org[7,1] = 'Rene'
        self.org[8,1] = 'Ritz'
        self.org[9,1] = 'Andy'
        self.org[:,2] = 'Wilbur', 'Andy', 'Kenneth', 'Rene', 'Jem', 'Micholo', 'Jayson', 'Imang', 'Rhei', 'Kim'
        self.org[:,3] = 'Ritz', 'Jholeeh', 'Micholo', 'Yego', 'Atchong', 'LJ', 'Jai', 'Jayson', 'Imang', 'Rhei'
        self.role_repeat_weeks = 2
        self.speak_repeat_weeks = 8
        self.maxiters = 25
                
    def generate_senior_speakers(self):
        # Fill speakers block
        ii= rand.choice(4, len(self.seniors))
        jj = rand.choice(12, len(self.seniors))
        k = 0
        for i in ii:
            for j in jj:
                if len(self.org[i,j]) >= 2:
                    continue
                else:
                    self.org[i,j] = self.seniors[k]
                    k += 1
                    if k > 10:
                        break
                if k > 10:
                    break
            if k > 10:
                break
                
    def generate_roles(self):
        for j in range(len(self.dates)):
            for i in range(len(self.roles)):                
                if len(self.org[i,j]) >=2:
                    continue
                    
                n = rand.choice(len(self.mems))
                name = self.mems[n]
                tag = self.tags[n]
                
                if i < 4:
                    n = rand.choice(len(self.juniors))
                    name = self.juniors[n]
                    speaker_condition = name not in self.org[:4, :] if j < self.speak_repeat_weeks \
                                        else name not in self.org[:4, j-self.speak_repeat_weeks:]
                    count = 1
                    self.role_repeat_weeks = 2
                    self.speak_repeat_weeks = 8
                    while speaker_condition is False:
                        if count%self.maxiters == 0:
                            self.role_repeat_weeks -= 1
                            self.speak_repeat_weeks -= 1
                        n = rand.choice(len(self.juniors))
                        name = self.juniors[n]
                        speaker_condition = name not in self.org[:4, :] if j < self.speak_repeat_weeks \
                                            else name not in self.org[:4, j-self.speak_repeat_weeks:]
                        count += 1
                    self.org[i,j] = name
                    continue
                        
                elif 3 < i < 8:
                    repeat_condition = name not in self.org[:, :] if j < self.role_repeat_weeks \
                                       else name not in self.org[:, j-self.role_repeat_weeks:]
                    count = 1
                    self.role_repeat_weeks = 2
                    self.speak_repeat_weeks = 8
                    while repeat_condition is False:
                        if count%self.maxiters == 0:
                            self.role_repeat_weeks -= 1
                            self.speak_repeat_weeks -= 1
                        n = rand.choice(len(self.mems))
                        name = self.mems[n]
                        tag = self.tags[n]
                        repeat_condition = name not in self.org[:, :] if j < self.role_repeat_weeks \
                                           else name not in self.org[:, j-self.role_repeat_weeks:]
                        count += 1
                    self.org[i,j] = name
                    continue
                
                else:
                    n = rand.choice(len(self.juniors))
                    name = self.juniors[n]
                    repeat_condition = name not in self.org[:, :] if j < self.role_repeat_weeks \
                                       else name not in self.org[:, j-self.role_repeat_weeks:]
                    count = 1
                    self.role_repeat_weeks = 2
                    self.speak_repeat_weeks = 8
                    while repeat_condition is False:
                        if count%self.maxiters == 0:
                            self.role_repeat_weeks -= 1
                            self.speak_repeat_weeks -= 1
                        n = rand.choice(len(self.juniors))
                        name = self.juniors[n]
                        repeat_condition = name not in self.org[:, :] if j < self.role_repeat_weeks \
                                           else name not in self.org[:, j-self.role_repeat_weeks:]
                        count += 1
                    self.org[i,j] = name
                    continue
                
    def make_df(self, **kwargs):
        self.df = pd.DataFrame(data=self.org,
                               index=self.roles,
                               columns=self.dates,
                               dtype=str,
                               **kwargs)
        
    def save(self, savename, fmt, **kwargs):
        if fmt == 'csv':
            self.df.to_csv(savename + '.csv', **kwargs)
        elif fmt == 'html':
            self.df.to_html(savename + '.html', **kwargs)
        else:
            raise NotImplementedError

In [3]:
mems, tags = np.genfromtxt('mems.txt', dtype=str,
                           unpack=True, skip_header=1, delimiter=',')
start = dt.datetime(2019, 9, 9)
end = dt.datetime(2019, 12, 2)
roles = pd.Series(['Speaker1', 'Speaker2', 'Speaker3', 'Speaker4',
                   'Eval1', 'Eval2', 'Eval3', 'Eval4',
                   'SeminarMaster', 'Timer'],
                  dtype='category')

sched = SeminarSched(mems, tags, roles)
sched.generate_senior_speakers()
sched.generate_roles()
sched.make_df()
# sched.save('sem2-1920', 'csv', index=False)
# sched.save('sem2-1920', 'html', index=False)
sched.df

,20-Jan,27-Jan,03-Feb,10-Feb,17-Feb,24-Feb,02-Mar,09-Mar,16-Mar,23-Mar,30-Mar,06-Apr,13-Apr,20-Apr,27-Apr,04-May,11-May
Speaker1,Jing,Imang,Wilbur,Ritz,Jara,LJ,Ritz,Andrew,Alfred,Kim,Jai,Ralph,Rene,Jem,LJ,Kim,Jem
Speaker2,Atchong,Jai,Andy,Jholeeh,Jholeeh,Imang,Clauzette,Micah,Mara,Jem,LJ,Ernest,Kim,Andy,Kenneth,Jayson,Jai
Speaker3,Rhei,Raymond,Kenneth,Micholo,Kim,Jai,Kenneth,Wilbur,Rene,Imang,Andy,Kenneth,Jayson,Wilbur,Imang,Rene,Wilbur
Speaker4,LJ,Jayson,Rene,Yego,Jem,Jayson,Andy,Micholo,Elexis,Jayson,Wilbur,Micholo,Imang,Jai,Micholo,Andy,LJ
Eval1,Andy,Elexis,Jem,Atchong,Jing,Andy,Jai,Kenneth,Jing,Micholo,Kim,Alfred,Clauzette,Imang,Jayson,Wilbur,LJ
Eval2,Jara,Jholeeh,Micholo,LJ,Paul,Atchong,Jholeeh,Paul,Kim,Andy,Paul,Elexis,Jing,Andrew,Ritz,Clauzette,Elexis
Eval3,Clauzette,Micholo,Jayson,Jai,Kenneth,Yego,Rene,Jing,Yego,Clauzette,Andrew,Mara,Micah,Paul,Ernest,Jara,Paul
Eval4,Yego,Rene,Imang,Jayson,Wilbur,Kenneth,Kim,Atchong,Ritz,Jai,Jholeeh,Ritz,Jara,LJ,Yego,Jholeeh,Andrew
SeminarMaster,None,Ritz,Rhei,Imang,Rene,Jem,Wilbur,LJ,Jayson,Wilbur,Rene,Imang,Wilbur,Kenneth,Rene,Jai,Imang
Timer,None,Andy,Kim,Rhei,Andy,Micholo,Jem,Imang,Jem,Kenneth,Jayson,Jem,Andy,Micholo,Kim,Jem,Micholo
